<a href="https://colab.research.google.com/github/ggeorgekkariuki/Natural-Language-Processing/blob/main/NLP_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17473536/17464789 [==============================] - 0s 0us/step


###More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:
- if the review is greater than 250 words then trim off the extra words
- if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

In [3]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [6]:
train_data.shape

(250,)

(25000, 250)

###Creating the Model
Now it's time to create the model. 

We'll use a word embedding layer as the first layer in our model and add a LongShortTermMemory layer afterwards that feeds into a dense node to get our predicted sentiment. 

32 stands for the output dimension of the vectors generated by the embedding layer. 

In [8]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Embedding(VOCAB_SIZE, 32),
     tf.keras.layers.LSTM(32),
     tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


###Training

In [9]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

Train

In [10]:
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 71s 110ms/step - loss: 0.4441 - acc: 0.7875 - val_loss: 0.3493 - val_acc: 0.8526
Epoch 2/10
625/625 [==============================] - 69s 110ms/step - loss: 0.2071 - acc: 0.9215 - val_loss: 0.3098 - val_acc: 0.8690
Epoch 3/10
625/625 [==============================] - 68s 108ms/step - loss: 0.1071 - acc: 0.9648 - val_loss: 0.3587 - val_acc: 0.8580
Epoch 4/10
625/625 [==============================] - 68s 109ms/step - loss: 0.0595 - acc: 0.9805 - val_loss: 0.4277 - val_acc: 0.8714
Epoch 5/10
625/625 [==============================] - 68s 109ms/step - loss: 0.0400 - acc: 0.9872 - val_loss: 0.5179 - val_acc: 0.8612
Epoch 6/10
625/625 [==============================] - 67s 108ms/step - loss: 0.0226 - acc: 0.9934 - val_loss: 0.5152 - val_acc: 0.8374
Epoch 7/10
625/625 [==============================] - 67s 108ms/step - loss: 0.0281 - acc: 0.9914 - val_loss: 0.6126 - val_acc: 0.8448
Epoch 8/10
625/625 [==============================] - 6

Evaluation

In [11]:
results = model.evaluate(test_data, test_labels)
results

782/782 [==============================] - 16s 21ms/step - loss: 0.5952 - acc: 0.8380


[0.5951875448226929, 0.8380399942398071]

###Making Predictions
Now let’s use our network to make predictions on our own reviews. 

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.




In [22]:
word_index = imdb.get_word_index()

def encode(text):
  token = tf.keras.preprocessing.text.text_to_word_sequence(text)
  token = [word_index[word] if word in word_index else 0 for word in token]
  return sequence.pad_sequences([token], MAXLEN)[0]


reverse_word_index = { value:key for key, value in word_index.items()}

def decode(sequence):
  PAD = 0
  text = ''
  for num in sequence:
    if num != PAD:
      text += reverse_word_index[num] + " "
  return text[:-1]

Testing the Encoding Function

In [23]:
pos_review = 'The new Spider Man movie was really amazing, I loved the special effects and the amazing experience.'

e = encode(pos_review)

print(e)

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [24]:
print(decode(e))

the new spider man movie was really amazing i loved the special effects and the amazing experience


####Make the prediction

In [25]:
def predict(text):
  encoded_text = encode(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

In [30]:
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.927183]
[0.24942583]
